# Concatinating .csv Files and Creating New df_subset with Improved Column Names

In [1]:
import csv
import glob
import pandas as pd

# Display all columns
pd.set_option('display.max_columns', None)

# Display all rows
pd.set_option('display.max_rows', None)

# Reads all csv files in this folder and concatenates them
csv_files = glob.glob('*.csv')
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Dictionary of column names to change
cols_to_rename = {
    "tourney_id": "tournament_id",
    "tourney_name": "tournament_name",
    "tourney_date": "tournament_date",
    "winner_ht": "p1_height",
    "winner_id": "p1_id",
    "winner_name": "p1_name",
    "winner_age": "p1_age",
    "loser_age": "p2_age", 
    "winner_hand": "p1_hand",
    "loser_ht": "p2_height",
    "loser_id": "p2_id",
    "loser_name": "p2_name",
    "loser_hand": "p2_hand",
    "tourney_level": "tournament_level",
}

# Rename useful columns
df.rename(columns=cols_to_rename, inplace=True)

# remove Round Robin (RR) and Bronze medal (BR) round rows
df = df.loc[~df['round'].isin(['RR', 'BR'])].copy()

# Change format of tournament date column
df['tournament_date'] = pd.to_datetime(df['tournament_date'], format='%Y%m%d')

# Define round order
round_order = ["Q1", "Q2", "Q3", "R128", "R64", "R32", "R16", "QF", "SF", "F"]

# Create an ordered categorical column for tournament round
df["round"] = pd.Categorical(df["round"], categories=round_order, ordered=True)

# Build the list of sort keys that are actually present, then sort
sort_keys = [c for c in ["tournament_date", "tournament_id", "round"] if c in df.columns]
df = df.sort_values(sort_keys, kind="mergesort").reset_index(drop=True)

In [2]:
# Create subset of dataframe with useful columns
df_subset = df[['tournament_date','tournament_name', 'tournament_level', 'surface', 'p1_name', 'p1_id', 'p1_age', 'p1_height', 'p1_hand', 'p2_name', 'p2_id', 'p2_age', 'p2_height', 'p2_hand']]
df_subset = df_subset.copy()

# Cleaning Data

### Looking for Anomalous Data

In [3]:
# Displays number of rows in a dataframe
len(df_subset) 

139636

In [4]:
# Displayes number of NaN values for each column
df_subset.isna().sum()

tournament_date         0
tournament_name         0
tournament_level        0
surface                 0
p1_name                 0
p1_id                   0
p1_age                  8
p1_height            6093
p1_hand                 0
p2_name                 0
p2_id                   0
p2_age                107
p2_height           13165
p2_hand                 7
dtype: int64

In [5]:
df_subset.describe()

,tournament_date,p1_id,p1_age,p1_height,p2_id,p2_age,p2_height
count,139636,139636.000000,139628.000000,133543.000000,139636.000000,139529.000000,126471.000000
mean,2018-10-29 09:45:54.991549184,125900.833302,26.129899,185.578376,127072.905712,25.977125,185.160772
min,2012-01-01 00:00:00,100644.000000,14.700000,3.000000,100644.000000,14.300000,3.000000
25%,2015-09-28 00:00:00,105091.000000,22.900000,180.000000,105173.000000,22.700000,180.000000
50%,2018-08-27 00:00:00,106045.000000,25.800000,185.000000,106109.000000,25.700000,185.000000
75%,2022-05-23 00:00:00,126214.000000,29.000000,191.000000,126536.000000,28.900000,190.000000
max,2024-12-18 00:00:00,213178.000000,48.900000,211.000000,213686.000000,53.900000,211.000000
std,NaN,36500.177381,4.251628,7.313518,37178.912213,4.411224,7.990879


## Anomolous Surface Data

In [6]:
df_subset[df_subset['surface'].isna()]

,tournament_date,tournament_name,tournament_level,surface,p1_name,p1_id,p1_age,p1_height,p1_hand,p2_name,p2_id,p2_age,p2_height,p2_hand


###### There are no rows with missing values in the surface column.

## Anomolous Age Data

In [9]:
df_subset[(df_subset['p1_age'].isna()) | (df_subset['p2_age'].isna())]

,tournament_date,tournament_name,tournament_level,surface,p1_name,p1_id,p1_age,p1_height,p1_hand,p2_name,p2_id,p2_age,p2_height,p2_hand
5291,2012-07-16,Gstaad,A,Clay,Denis Gremelmayr,103823,30.9,183.0,L,Raphael Mori,110598,NaN,NaN,U
5426,2012-07-16,Atlanta,A,Hard,Tim Smyczek,105065,24.5,175.0,R,Judd Motz,111157,NaN,NaN,U
7199,2012-09-10,Petange CH,C,Hard,Jan Hernych,103401,33.1,190.0,R,Maciej Najfeld,111169,NaN,NaN,U
18855,2014-01-27,Chitre CH,C,Hard,Juan Ignacio Londero,106228,20.4,180.0,R,Jose Carlos Peralta,144655,NaN,NaN,U
37073,2016-01-11,Bangkok CH,C,Hard,Alexander Vasilenko,111193,20.8,NaN,R,Jirayu Limpathum,127151,NaN,NaN,U
37516,2016-01-18,Rio de Janeiro CH,C,Clay,Alexandre Girotto,127132,18.6,NaN,U,Joao Luiz Carvalho,206817,NaN,NaN,U
40799,2016-05-02,Rome CH,C,Clay,Vladimir Uzhylovsky,105130,28.1,196.0,R,Gianmarco De Santis,206836,NaN,NaN,NaN
41425,2016-05-16,Mestre CH,C,Clay,Thomas Giraudeau,105875,24.4,NaN,U,Filippo Moggian,206840,NaN,NaN,NaN
43042,2016-06-27,Marburg CH,C,Clay,Gianluca Mager,126149,21.5,188.0,R,Beruk Tsegai,206855,NaN,NaN,NaN
43190,2016-07-04,Todi CH,C,Clay,Viktor Galovic,105634,25.7,193.0,R,Vittorio Angeli,206857,NaN,NaN,U


###### All of the players that have missing p1_age and p2_age values also have missing height data. I can not find these player's ages or heights on the ATP website. There are a total of 115 missing age entries in a data frame of 143272 so these will be removed from the df_subset.

In [10]:
df_subset['p1_age'].describe()

count    139628.000000
mean         26.129899
std           4.251628
min          14.700000
25%          22.900000
50%          25.800000
75%          29.000000
max          48.900000
Name: p1_age, dtype: float64

In [11]:
df_subset['p2_age'].describe()

count    139529.000000
mean         25.977125
std           4.411224
min          14.300000
25%          22.700000
50%          25.700000
75%          28.900000
max          53.900000
Name: p2_age, dtype: float64

In [12]:
df_subset[(df_subset['p1_age'] < 16) | (df_subset['p2_age'] < 16)]

,tournament_date,tournament_name,tournament_level,surface,p1_name,p1_id,p1_age,p1_height,p1_hand,p2_name,p2_id,p2_age,p2_height,p2_hand
1698,2012-03-05,Santiago CH,C,Clay,Fernando Romboli,105262,23.1,180.0,R,Cristian Garin,106426,15.7,185.0,R
7879,2012-10-07,Shanghai Masters,M,Hard,Brian Baker,104548,27.4,190.0,R,Zhizhen Zhang,111190,15.9,193.0,R
12410,2013-05-13,Busan CH,C,Hard,Di Wu,105842,21.6,175.0,R,Duck Hee Lee,117353,14.9,175.0,R
13931,2013-07-08,Newport,A,Grass,Michal Przysiezny,104308,29.3,185.0,R,Stefan Kozlov,111578,15.4,183.0,R
14545,2013-07-22,Astana CH,C,Hard,Dane Propoggia,105525,23.2,NaN,R,Alexey Nesterov,106347,15.4,NaN,U
14576,2013-07-22,Umag,A,Clay,Nikola Mektic,105254,24.5,183.0,R,Andres Arce,126121,15.6,NaN,U
14806,2013-07-28,Kitzbuhel,A,Clay,Dennis Novak,110602,19.9,183.0,R,Matthias Haim,126147,15.5,NaN,R
16327,2013-09-16,Campinas CH,C,Clay,Gastao Elias,105671,22.8,180.0,R,Ghilherme Scarpelli,126230,15.6,NaN,U
16921,2013-10-07,Tashkent CH,C,Hard,Marco Chiudinelli,103843,32.0,185.0,R,Jurabek Karimov,127300,15.3,NaN,R
17195,2013-10-14,Moscow,A,Hard,Egor Gerasimov,106078,20.9,196.0,R,Andrey Rublev,126094,15.9,188.0,R


In [13]:
df_subset[(df_subset['p1_age'] > 45) | (df_subset['p2_age'] > 45)]

,tournament_date,tournament_name,tournament_level,surface,p1_name,p1_id,p1_age,p1_height,p1_hand,p2_name,p2_id,p2_age,p2_height,p2_hand
22587,2014-06-15,s Hertogenbosch,A,Grass,Adrien Bossel,104847,27.4,NaN,L,Sander Groen,101492,45.9,190.0,L
39089,2016-03-14,Guadalajara CH,C,Hard,Mohamed Safwat,105633,25.4,180.0,R,Agustin Moreno,101339,48.9,180.0,R
39292,2016-03-21,San Luis Potosi CH,C,Clay,Agustin Moreno,101339,48.9,180.0,R,Adam El Mihdawy,105398,26.6,NaN,U
39305,2016-03-21,San Luis Potosi CH,C,Clay,Caio Zampieri,104738,29.8,183.0,R,Agustin Moreno,101339,48.9,180.0,R
39545,2016-03-28,Leon CH,C,Hard,Antonio Ruiz Rosales,104431,31.5,NaN,R,Agustin Moreno,101339,48.9,180.0,R
43774,2016-07-18,Binghamton CH,C,Hard,Winston Lin,122241,23.2,NaN,U,Charlie Maher,132430,53.9,NaN,L
56747,2017-08-07,Aptos CH,C,Hard,Miguel Diaz,127139,37.7,NaN,U,Rick Kepler,108094,50.4,NaN,R
60916,2017-11-20,Rio De Janeiro CH,C,Clay,Fabricio Neis,105571,27.4,NaN,U,George Hime,124263,49.1,NaN,L
62197,2018-02-05,San Francisco CH,C,Hard,Sem Verbeek,200465,23.8,193.0,L,Jeff Greenwald,101305,51.2,NaN,R
122062,2023-08-28,Zhangjiagang CH,C,Hard,Ray Ho,202052,23.5,188.0,L,Toshihide Matsui,103175,45.3,178.0,R


###### I have checked for anomalously old and young players. All of the very young and old players listed in the dataset have their correct ages listed. 

## Anomolous Height Data

In [14]:
# Total number of rows with missing height data for eiter p1 or p2 
len(df_subset[(df_subset['p1_height'].isna()) | (df_subset['p2_height'].isna())])

17296

###### There are 17296 rows in the dataset with missing height data.

In [15]:
# Create list of players with missing height data
missing_height_p1_names = df_subset[df_subset['p1_height'].isna()]['p1_name']
missing_height_p2_names = df_subset[df_subset['p2_height'].isna()]['p2_name']
missing_height_names = set(missing_height_p1_names) | set(missing_height_p2_names)

In [17]:
import wptools
import re

def get_heights(names): 
    # store results as name: height_cm
    heights = {}  
    
    for name in names:
        try:
            page = wptools.page(name, silent=True).get_parse() 
            infobox = page.data.get('infobox', {}) 
            height_raw = infobox.get('height', '') 
            
            # Extract height in meters from the wikipedia page
            match = re.search(r'\{\{height\|m\|=\|([\d.]+)\}\}', height_raw) 
            if match: 
                height_cm = float(match.group(1)) * 100
                heights[name] = height_cm
            else:
                # If it could not be parsed
                heights[name] = None  
        except Exception:
            # page not found or other error
            heights[name] = None  
    
    return heights

#get_heights(missing_height_names)

ModuleNotFoundError: No module named 'wptools'

###### I have searched wikipedia for their height data to amend the entries, but these players do not have wikipedia pages. The NaN results will not show in later in seaborn plots, however these results will cause errors later on when trying to train a model on this data so they will be removed. There are 17296 rows being deleted which may impact ELO calculations, therefore I will remove these rows after the ELOs have been calculated.    

In [18]:
df_subset['p1_height'].describe()

count    133543.000000
mean        185.578376
std           7.313518
min           3.000000
25%         180.000000
50%         185.000000
75%         191.000000
max         211.000000
Name: p1_height, dtype: float64

In [19]:
df_subset['p2_height'].describe()

count    126471.000000
mean        185.160772
std           7.990879
min           3.000000
25%         180.000000
50%         185.000000
75%         190.000000
max         211.000000
Name: p2_height, dtype: float64

###### In both p1 and p2 height columns, there is a player listed as being 3 cm tall, this is clearly anomolous data so I will find their names and try to amend their height data.

In [20]:
# Checking for anomolously short players
df_subset[(df_subset['p1_height'] <= 160) | (df_subset['p2_height'] <= 160)]

,tournament_date,tournament_name,tournament_level,surface,p1_name,p1_id,p1_age,p1_height,p1_hand,p2_name,p2_id,p2_age,p2_height,p2_hand
4818,2012-07-02,Lima CH,C,Clay,Marcelo Demoliner,105269,23.4,191.0,R,Jorge Brian Panta Herreros,106410,16.9,3.0,R
17994,2013-11-11,Lima CH,C,Clay,Martin Alund,104651,27.8,183.0,R,Jorge Brian Panta Herreros,106410,18.3,3.0,R
23853,2014-07-21,Umag,A,Clay,Marco Cecchinato,106065,21.8,185.0,R,Ilija Vucic,105661,23.7,145.0,R
27106,2014-11-17,Lima CH,C,Clay,Roberto Carballes Baena,106148,21.6,183.0,R,Jorge Brian Panta Herreros,106410,19.3,3.0,R
42574,2016-06-13,Poprad CH,C,Clay,Ilija Vucic,105661,25.6,145.0,R,Tomas Vyrostko,129147,29.0,NaN,R
42582,2016-06-13,Poprad CH,C,Clay,Jurgen Melzer,103781,35.0,183.0,L,Ilija Vucic,105661,25.6,145.0,R
43244,2016-07-04,Cali CH,C,Clay,Jorge Brian Panta Herreros,106410,20.9,3.0,R,Juan Montes,124206,35.1,NaN,U
43259,2016-07-04,Cali CH,C,Clay,Matias Zukas,125825,20.1,183.0,R,Jorge Brian Panta Herreros,106410,20.9,3.0,R
44684,2016-08-08,Portoroz CH,C,Hard,Cem Ilkel,117356,20.9,185.0,R,Ilija Vucic,105661,25.7,145.0,R
44964,2016-08-15,Cordenons CH,C,Clay,Fabrizio Ornago,122058,24.3,NaN,U,Ilija Vucic,105661,25.8,145.0,R


In [21]:
# Checking for anomolously tall players
df_subset[(df_subset['p1_height'] >= 211) | (df_subset['p2_height'] >= 211)]

,tournament_date,tournament_name,tournament_level,surface,p1_name,p1_id,p1_age,p1_height,p1_hand,p2_name,p2_id,p2_age,p2_height,p2_hand
32632,2015-07-20,Binghamton CH,C,Hard,Sekou Bangoura,105871,23.6,183.0,R,Reilly Opelka,124187,17.8,211.0,R
32937,2015-07-27,Atlanta,A,Hard,Shuichi Sekiguchi,105810,24.0,168.0,R,Reilly Opelka,124187,17.9,211.0,R
33739,2015-08-31,US Open,G,Hard,Reilly Opelka,124187,18.0,211.0,R,Christian Lindell,105872,23.7,193.0,R
33790,2015-08-31,US Open,G,Hard,Kimmer Coppejans,106293,21.5,178.0,R,Reilly Opelka,124187,18.0,211.0,R
39297,2016-03-21,San Luis Potosi CH,C,Clay,Reilly Opelka,124187,18.5,211.0,R,Laurynas Grigelis,105827,24.6,183.0,R
39308,2016-03-21,San Luis Potosi CH,C,Clay,Robin Stanek,106377,21.2,NaN,L,Reilly Opelka,124187,18.5,211.0,R
39741,2016-04-04,Houston,A,Clay,Reilly Opelka,124187,18.6,211.0,R,Dennis Novikov,106261,22.4,193.0,R
39748,2016-04-04,Houston,A,Clay,Matthew Barton,105881,24.2,191.0,R,Reilly Opelka,124187,18.6,211.0,R
39754,2016-04-04,Houston,A,Clay,Sam Querrey,105023,28.4,198.0,R,Reilly Opelka,124187,18.6,211.0,R
43352,2016-07-11,Newport,A,Grass,Frank Dancevic,104433,31.7,185.0,R,Reilly Opelka,124187,18.8,211.0,R


###### I looked up the players with anonomous height data i.e. height = 3 cm, their heights are not shown on the ATP website so they will be removed after calculating ELOs etc.

## Missing Player Hand Data

In [22]:
df_subset[(df_subset['p1_hand'] == 'U') | (df_subset['p2_hand'] == 'U')].describe()

,tournament_date,p1_id,p1_age,p1_height,p2_id,p2_age,p2_height
count,6769,6769.000000,6762.000000,4290.000000,6769.000000,6673.00000,1319.000000
mean,2017-07-20 16:23:28.302555648,122781.813562,24.998462,184.151981,134212.970306,23.65834,184.170584
min,2012-01-01 00:00:00,101339.000000,14.700000,3.000000,102093.000000,14.30000,3.000000
25%,2015-09-07 00:00:00,105299.000000,22.100000,180.000000,105640.000000,20.50000,180.000000
50%,2017-01-30 00:00:00,106072.000000,24.700000,185.000000,111765.000000,23.50000,185.000000
75%,2018-10-08 00:00:00,125796.000000,27.600000,188.000000,138758.000000,26.20000,188.000000
max,2024-11-25 00:00:00,213178.000000,48.900000,211.000000,213673.000000,53.90000,211.000000
std,NaN,32661.845366,3.969592,9.469453,39446.394762,4.15629,8.549358


###### Players with unknown handedness usually have other missing data so they will be removed after calculating ELOs etc.

# Calculating New Features for Dataframe

## Calculating Age Differences

In [23]:
# Create empty lists to store age differences
p1_age_diff_list = []
p2_age_diff_list = []

# loops over the age columns and calculates players age difference
for p1, p2 in zip(df_subset['p1_age'], df_subset['p2_age']):
    p1_age_diff = p1 - p2
    p2_age_diff = p2 - p1

    # Adds calculated age differences to list
    p1_age_diff_list.append(p1_age_diff)
    p2_age_diff_list.append(p2_age_diff)

# Assign lists to new df columns
df_subset['p1_age_diff'] = p1_age_diff_list
df_subset['p2_age_diff'] = p2_age_diff_list
df_subset['p1_age_diff'] = df_subset['p1_age_diff'].round(1)
df_subset['p2_age_diff'] = df_subset['p2_age_diff'].round(1)

## Calculating Height Differences

In [24]:
# Create empty lists to store height differences
p1_height_diff_list = []
p2_height_diff_list = []

# loops over the height columns and calculates players height difference
for p1, p2 in zip(df_subset['p1_height'], df_subset['p2_height']):
    p1_height_diff = p1 - p2
    p2_height_diff = p2 - p1

    # Adds calculated age differences to list
    p1_height_diff_list.append(p1_height_diff)
    p2_height_diff_list.append(p2_height_diff)

# Assign lists to new df columns
df_subset['p1_height_diff'] = p1_height_diff_list
df_subset['p2_height_diff'] = p2_height_diff_list

# Calculating Previous H2H Wins Against Opponent Columns

In [26]:
from collections import defaultdict 

# Dictionary that stores head-to-head match results. Returns 0 if the key does not exist (players have never played before)
h2h_wins_dict = defaultdict(int)

# Dictionary to store sequence of winners of matches between players
h2h_history_dict = defaultdict(list)

# Stores total h2h wins before the current match
p1_h2h_wins_before = []
p2_h2h_wins_before = []

# Stores total h2h win difference before the current match
p1_h2h_wins_total_diff_before = []
p2_h2h_wins_total_diff_before = []

# Stores h2h win difference in last game
p1_h2h_wins_last1_diff_before = []
p2_h2h_wins_last1_diff_before = []

# Stores h2h win difference in last 2 games
p1_h2h_wins_last2_diff_before = []
p2_h2h_wins_last2_diff_before = []

# Stores h2h win difference in last 3 games
p1_h2h_wins_last3_diff_before = []
p2_h2h_wins_last3_diff_before = []

# Stores h2h win difference in last 4 games
p1_h2h_wins_last4_diff_before = []
p2_h2h_wins_last4_diff_before = []

# Stores h2h win difference in last 5 games
p1_h2h_wins_last5_diff_before = []
p2_h2h_wins_last5_diff_before = []

# Stores h2h win difference in last 10 games
p1_h2h_wins_last10_diff_before = []
p2_h2h_wins_last10_diff_before = []


# Iterate through each match in df_subset, returning player_ids as pairs  
for p1, p2 in zip(df_subset['p1_id'], df_subset['p2_id']):
    
    # Creates keys for head-to-head matches
    wins_key1 = (p1, p2)
    wins_key2 = (p2, p1)
    wins_match_key = tuple(sorted([p1, p2]))

    # Get total wins for each player before this match
    p1_h2h_wins = h2h_wins_dict[wins_key1]
    p2_h2h_wins = h2h_wins_dict[wins_key2]
    
    # Saves wins to respective p1_h2h_wins_before and p2_h2h_wins_before lists to then be used for dataframe columns
    p1_h2h_wins_before.append(p1_h2h_wins)
    p2_h2h_wins_before.append(p2_h2h_wins)
    
    # Calculates wins difference
    p1_h2h_wins_diff = p1_h2h_wins - p2_h2h_wins
    p2_h2h_wins_diff = p2_h2h_wins - p1_h2h_wins
    
    # Saves differences to px_h2h_wins_diff_before lists to then be used for dataframe columns
    p1_h2h_wins_total_diff_before.append(p1_h2h_wins_diff)
    p2_h2h_wins_total_diff_before.append(p2_h2h_wins_diff)

    # Get last 1 match results and compute difference
    history_last1 = h2h_history_dict[wins_match_key][-1:]
    p1_last1_wins = history_last1.count(p1)
    p2_last1_wins = history_last1.count(p2)
    p1_last1_diff = p1_last1_wins - p2_last1_wins
    p2_last1_diff = p2_last1_wins - p1_last1_wins
    p1_h2h_wins_last1_diff_before.append(p1_last1_diff)
    p2_h2h_wins_last1_diff_before.append(p2_last1_diff)

    # Get last 2 match results and compute difference
    history_last2 = h2h_history_dict[wins_match_key][-2:]
    p1_last2_wins = history_last2.count(p1)
    p2_last2_wins = history_last2.count(p2)
    p1_last2_diff = p1_last2_wins - p2_last2_wins
    p2_last2_diff = p2_last2_wins - p1_last2_wins
    p1_h2h_wins_last2_diff_before.append(p1_last2_diff)
    p2_h2h_wins_last2_diff_before.append(p2_last2_diff)

    # Get last 3 match results and compute difference
    history_last3 = h2h_history_dict[wins_match_key][-3:]
    p1_last3_wins = history_last3.count(p1)
    p2_last3_wins = history_last3.count(p2)
    p1_last3_diff = p1_last3_wins - p2_last3_wins
    p2_last3_diff = p2_last3_wins - p1_last3_wins
    p1_h2h_wins_last3_diff_before.append(p1_last3_diff)
    p2_h2h_wins_last3_diff_before.append(p2_last3_diff)

    # Get last 4 match results and compute difference
    history_last4 = h2h_history_dict[wins_match_key][-4:]
    p1_last4_wins = history_last4.count(p1)
    p2_last4_wins = history_last4.count(p2)
    p1_last4_diff = p1_last4_wins - p2_last4_wins
    p2_last4_diff = p2_last4_wins - p1_last4_wins
    p1_h2h_wins_last4_diff_before.append(p1_last4_diff)
    p2_h2h_wins_last4_diff_before.append(p2_last4_diff)

    # Get last 5 match results and compute difference
    history_last5 = h2h_history_dict[wins_match_key][-5:]
    p1_last5_wins = history_last5.count(p1)
    p2_last5_wins = history_last5.count(p2)
    p1_last5_diff = p1_last5_wins - p2_last5_wins
    p2_last5_diff = p2_last5_wins - p1_last5_wins
    p1_h2h_wins_last5_diff_before.append(p1_last5_diff)
    p2_h2h_wins_last5_diff_before.append(p2_last5_diff)

    # Get last 10 match results and compute difference
    history_last10 = h2h_history_dict[wins_match_key][-10:]
    p1_last10_wins = history_last10.count(p1)
    p2_last10_wins = history_last10.count(p2)
    p1_last10_diff = p1_last10_wins - p2_last10_wins
    p2_last10_diff = p2_last10_wins - p1_last10_wins
    p1_h2h_wins_last10_diff_before.append(p1_last10_diff)
    p2_h2h_wins_last10_diff_before.append(p2_last10_diff)

    # Player 1 always wins in this df_subset, updates head-to-head
    h2h_wins_dict[wins_key1] += 1  

    # Updates h2h history dicitonary 
    h2h_history_dict[wins_match_key].append(p1)

# Assign to dataframe
df_subset['p1_h2h_wins'] = p1_h2h_wins_before
df_subset['p2_h2h_wins'] = p2_h2h_wins_before
df_subset['p1_h2h_wins_before_total_diff'] = p1_h2h_wins_total_diff_before
df_subset['p2_h2h_wins_before_total_diff'] = p2_h2h_wins_total_diff_before
df_subset['p1_h2h_wins_before_last1_diff'] = p1_h2h_wins_last1_diff_before
df_subset['p2_h2h_wins_before_last1_diff'] = p2_h2h_wins_last1_diff_before
df_subset['p1_h2h_wins_before_last2_diff'] = p1_h2h_wins_last2_diff_before
df_subset['p2_h2h_wins_before_last2_diff'] = p2_h2h_wins_last2_diff_before
df_subset['p1_h2h_wins_before_last3_diff'] = p1_h2h_wins_last3_diff_before
df_subset['p2_h2h_wins_before_last3_diff'] = p2_h2h_wins_last3_diff_before
df_subset['p1_h2h_wins_before_last4_diff'] = p1_h2h_wins_last4_diff_before
df_subset['p2_h2h_wins_before_last4_diff'] = p2_h2h_wins_last4_diff_before
df_subset['p1_h2h_wins_before_last5_diff'] = p1_h2h_wins_last5_diff_before
df_subset['p2_h2h_wins_before_last5_diff'] = p2_h2h_wins_last5_diff_before
df_subset['p1_h2h_wins_before_last10_diff'] = p1_h2h_wins_last10_diff_before
df_subset['p2_h2h_wins_before_last10_diff'] = p2_h2h_wins_last10_diff_before

# Calculating ELO

In [27]:
from collections import defaultdict

# Dictionary that stores players ELO. Returns 1500 if the key does not exist (player has not played before)
elo_dict = defaultdict(lambda: 1500)

# Stores immediate elo
p1_elo_before = []
p2_elo_before = []

# Iterate through each match in df_subset, returning player_ids as pairs 
for p1, p2 in zip(df_subset['p1_id'], df_subset['p2_id']):
    elo_key1 = p1
    elo_key2 = p2

    # Checks dictionary for ELO and stores the ELO as px_elo
    p1_elo = elo_dict[elo_key1]
    p2_elo = elo_dict[elo_key2]

    # Adds ELO to the list px_elo_before
    p1_elo_before.append(p1_elo)
    p2_elo_before.append(p2_elo)

    # Calculates expected score
    p1_expected_score = 1 / (1 + 10**((p2_elo - p1_elo)/400))
    p2_expected_score = 1 / (1 + 10**((p1_elo - p2_elo)/400))

    # Calculates ELO after the match
    K = 32
    p1_elo_after = int(p1_elo + K * (1 - p1_expected_score))
    p2_elo_after = int(p2_elo + K * (0 - p2_expected_score))

    # Stores new ELO in the dictionary
    elo_dict[elo_key1] = p1_elo_after
    elo_dict[elo_key2] = p2_elo_after
    
# Assign to dataframe
df_subset['p1_elo_before'] = p1_elo_before
df_subset['p2_elo_before'] = p2_elo_before

#### Calculating Rolling ELO

In [29]:
df_subset['p1_elo_rolling_last5'] = (df_subset.groupby('p1_id')['p1_elo_before'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_elo_rolling_last5'] = (df_subset.groupby('p2_id')['p2_elo_before'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p1_elo_rolling_last10'] = (df_subset.groupby('p1_id')['p1_elo_before'].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_elo_rolling_last10'] = (df_subset.groupby('p2_id')['p2_elo_before'].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p1_elo_rolling_last20'] = (df_subset.groupby('p1_id')['p1_elo_before'].transform(lambda x: x.shift().rolling(20, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_elo_rolling_last20'] = (df_subset.groupby('p2_id')['p2_elo_before'].transform(lambda x: x.shift().rolling(20, min_periods=1).mean())).fillna(1500).astype(int)

#### Calculating Rolling ELO difference

In [30]:
# Create empty lists to store rolling ELO differences
p1_elo_rolling_last5_diff_list = []
p2_elo_rolling_last5_diff_list = []

p1_elo_rolling_last10_diff_list = []
p2_elo_rolling_last10_diff_list = []

p1_elo_rolling_last20_diff_list = []
p2_elo_rolling_last20_diff_list = []

# Loops over the last5 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_elo_rolling_last5'], df_subset['p2_elo_rolling_last5']):
    p1_elo_rolling_last5_diff = p1 - p2
    p2_elo_rolling_last5_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_elo_rolling_last5_diff_list.append(p1_elo_rolling_last5_diff)
    p2_elo_rolling_last5_diff_list.append(p2_elo_rolling_last5_diff)

# Loops over the last10 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_elo_rolling_last10'], df_subset['p2_elo_rolling_last10']):
    p1_elo_rolling_last10_diff = p1 - p2
    p2_elo_rolling_last10_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_elo_rolling_last10_diff_list.append(p1_elo_rolling_last10_diff)
    p2_elo_rolling_last10_diff_list.append(p2_elo_rolling_last10_diff)

# Loops over the last20 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_elo_rolling_last20'], df_subset['p2_elo_rolling_last20']):
    p1_elo_rolling_last20_diff = p1 - p2
    p2_elo_rolling_last20_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_elo_rolling_last20_diff_list.append(p1_elo_rolling_last20_diff)
    p2_elo_rolling_last20_diff_list.append(p2_elo_rolling_last20_diff)


# Assign lists to new df columns
df_subset['p1_elo_rolling_last5_diff_before'] = p1_elo_rolling_last5_diff_list
df_subset['p2_elo_rolling_last5_diff_before'] = p2_elo_rolling_last5_diff_list
df_subset['p1_elo_rolling_last10_diff_before'] = p1_elo_rolling_last10_diff_list
df_subset['p2_elo_rolling_last10_diff_before'] = p2_elo_rolling_last10_diff_list
df_subset['p1_elo_rolling_last20_diff_before'] = p1_elo_rolling_last20_diff_list
df_subset['p2_elo_rolling_last20_diff_before'] = p2_elo_rolling_last20_diff_list

#### Calculating ELO difference

In [31]:
# Create empty lists to store ELO differences
p1_elo_diff_list = []
p2_elo_diff_list = []

# Loops over the ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_elo_before'], df_subset['p2_elo_before']):
    p1_elo_diff = p1 - p2
    p2_elo_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_elo_diff_list.append(p1_elo_diff)
    p2_elo_diff_list.append(p2_elo_diff)

# Assign lists to new df columns
df_subset['p1_elo_diff_before'] = p1_elo_diff_list
df_subset['p2_elo_diff_before'] = p2_elo_diff_list

#### Displays Total Number of Games Played on each Surface

In [32]:
surface_dict = {}

for surface in df_subset['surface']:
    surface_dict[surface] = surface_dict.get(surface, 0) + 1

print(surface_dict)

{'Hard': 73452, 'Clay': 57901, 'Carpet': 501, 'Grass': 7782}


#### Calculating surface ELO

In [33]:
from collections import defaultdict

# Creates a nested defaultdict: elo_dict[player_id][surface] = ELO
elo_dict = defaultdict(lambda: defaultdict(lambda: 1500))

# Lists to store ELOs before the match
p1_surface_elo_before = []
p2_surface_elo_before = []

# Loop through df_subset row by row
for surface, p1, p2 in zip(df_subset['surface'], df_subset['p1_id'], df_subset['p2_id']):

    # Get each player's current ELO on this surface
    p1_elo = elo_dict[p1][surface]
    p2_elo = elo_dict[p2][surface]

    # Store ELOs before match
    p1_surface_elo_before.append(p1_elo)
    p2_surface_elo_before.append(p2_elo)

    # Calculate expected scores
    p1_expected = 1 / (1 + 10 ** ((p2_elo - p1_elo) / 400))
    p2_expected = 1 / (1 + 10 ** ((p1_elo - p2_elo) / 400))

    # Update ELOs assuming p1 wins
    K = 32
    elo_dict[p1][surface] = int(p1_elo + K * (1 - p1_expected))
    elo_dict[p2][surface] = int(p2_elo + K * (0 - p2_expected))

# Add columns to df_subset
df_subset['p1_surface_elo_before'] = p1_surface_elo_before
df_subset['p2_surface_elo_before'] = p2_surface_elo_before

#### Calculating Rolling Surface ELO

In [34]:
df_subset['p1_surface_elo_rolling_last5'] = (df_subset.groupby('p1_id')['p1_surface_elo_before'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_surface_elo_rolling_last5'] = (df_subset.groupby('p2_id')['p2_surface_elo_before'].transform(lambda x: x.shift().rolling(5, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p1_surface_elo_rolling_last10'] = (df_subset.groupby('p1_id')['p1_surface_elo_before'].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_surface_elo_rolling_last10'] = (df_subset.groupby('p2_id')['p2_surface_elo_before'].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p1_surface_elo_rolling_last20'] = (df_subset.groupby('p1_id')['p1_surface_elo_before'].transform(lambda x: x.shift().rolling(20, min_periods=1).mean())).fillna(1500).astype(int)
df_subset['p2_surface_elo_rolling_last20'] = (df_subset.groupby('p2_id')['p2_surface_elo_before'].transform(lambda x: x.shift().rolling(20, min_periods=1).mean())).fillna(1500).astype(int)

#### Calculating Rolling Surface ELO difference

In [35]:
# Create empty lists to store rolling ELO differences
p1_surface_elo_rolling_last5_diff_list = []
p2_surface_elo_rolling_last5_diff_list = []

p1_surface_elo_rolling_last10_diff_list = []
p2_surface_elo_rolling_last10_diff_list = []

p1_surface_elo_rolling_last20_diff_list = []
p2_surface_elo_rolling_last20_diff_list = []

# Loops over the last5 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_surface_elo_rolling_last5'], df_subset['p2_surface_elo_rolling_last5']):
    p1_surface_elo_rolling_last5_diff = p1 - p2
    p2_surface_elo_rolling_last5_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_surface_elo_rolling_last5_diff_list.append(p1_surface_elo_rolling_last5_diff)
    p2_surface_elo_rolling_last5_diff_list.append(p2_surface_elo_rolling_last5_diff)

# Loops over the last10 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_surface_elo_rolling_last10'], df_subset['p2_surface_elo_rolling_last10']):
    p1_surface_elo_rolling_last10_diff = p1 - p2
    p2_surface_elo_rolling_last10_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_surface_elo_rolling_last10_diff_list.append(p1_surface_elo_rolling_last10_diff)
    p2_surface_elo_rolling_last10_diff_list.append(p2_surface_elo_rolling_last10_diff)

# Loops over the last20 rolling ELO columns and calculates the players ELO difference
for p1, p2 in zip(df_subset['p1_surface_elo_rolling_last20'], df_subset['p2_surface_elo_rolling_last20']):
    p1_surface_elo_rolling_last20_diff = p1 - p2
    p2_surface_elo_rolling_last20_diff = p2 - p1

    # Adds calculated ELO differences to list
    p1_surface_elo_rolling_last20_diff_list.append(p1_surface_elo_rolling_last20_diff)
    p2_surface_elo_rolling_last20_diff_list.append(p2_surface_elo_rolling_last20_diff)


# Assign lists to new df columns
df_subset['p1_surface_elo_rolling_last5_diff_before'] = p1_surface_elo_rolling_last5_diff_list
df_subset['p2_surface_elo_rolling_last5_diff_before'] = p2_surface_elo_rolling_last5_diff_list
df_subset['p1_surface_elo_rolling_last10_diff_before'] = p1_surface_elo_rolling_last10_diff_list
df_subset['p2_surface_elo_rolling_last10_diff_before'] = p2_surface_elo_rolling_last10_diff_list
df_subset['p1_surface_elo_rolling_last20_diff_before'] = p1_surface_elo_rolling_last20_diff_list
df_subset['p2_surface_elo_rolling_last20_diff_before'] = p2_surface_elo_rolling_last20_diff_list

#### Calculating surface ELO difference

In [36]:
# Creates empty lists to store surface ELO differences
p1_surface_elo_diff_list = []
p2_surface_elo_diff_list = []

# Loops over the surface ELO columns and calculates the surface ELO difference 
for p1, p2 in zip(df_subset['p1_surface_elo_before'], df_subset['p2_surface_elo_before']):
    p1_surface_elo_diff = p1 - p2
    p2_surface_elo_diff = p2 - p1

    # Adds calculates surface ELO difference to list
    p1_surface_elo_diff_list.append(p1_surface_elo_diff)
    p2_surface_elo_diff_list.append(p2_surface_elo_diff)

# Assign lists to new df columns
df_subset['p1_surface_elo_diff_before'] = p1_surface_elo_diff_list
df_subset['p2_surface_elo_diff_before'] = p2_surface_elo_diff_list

# Calculating Total Number of matches a Player Has Played

In [37]:
from collections import defaultdict

# Dictionary to store the number of matches each player has played before
total_matches_dict = defaultdict(int)

# Empty lists to store total number of matches played by each player 
p1_total_matches_before = []
p2_total_matches_before = []

# Loops over player id columns and creates a key based on the players id
for p1, p2 in zip(df_subset['p1_id'], df_subset['p2_id']):
    total_matches_key1 = p1
    total_matches_key2 = p2

    # Get current total number of matches for each player before current game
    p1_total_matches = total_matches_dict[total_matches_key1]
    p2_total_matches = total_matches_dict[total_matches_key2]

    # Stores the total number of matches for each player before current game
    p1_total_matches_before.append(p1_total_matches)
    p2_total_matches_before.append(p2_total_matches)

    # Increment the match count for each player
    p1_total_matches_after = p1_total_matches + 1
    p2_total_matches_after = p2_total_matches + 1

    # Update the dictionary with the new total matches played after this match 
    total_matches_dict[total_matches_key1] = p1_total_matches_after
    total_matches_dict[total_matches_key2] = p2_total_matches_after

# Assign lists to new df columns
df_subset['p1_total_matches_played_before'] = p1_total_matches_before
df_subset['p2_total_matches_played_before'] = p2_total_matches_before

#### Calculate total matches player difference

In [38]:
p1_total_career_matches_diff_list = []
p2_total_career_matches_diff_list = []

for p1, p2 in zip(df_subset['p1_total_matches_played_before'], df_subset['p2_total_matches_played_before']):
    p1_career_matches_diff = p1 - p2
    p2_career_matches_diff = p2 - p1

    p1_total_career_matches_diff_list.append(p1_career_matches_diff)
    p2_total_career_matches_diff_list.append(p2_career_matches_diff)

df_subset['p1_total_matches_played_before_diff'] = p1_total_career_matches_diff_list
df_subset['p2_total_matches_played_before_diff'] = p2_total_career_matches_diff_list

# Calculating Total Wins

In [39]:
from collections import defaultdict

# Initialize dictionaries to track career wins and match outcomes
career_wins_dict = defaultdict(int)
career_wins_history_dict = defaultdict(list)

# Lists to store computed features for Dataframe
p1_career_wins_before = []
p2_career_wins_before = []

# Player win percentage in last 3 matches before current match
p1_career_wins_last3_pct_before = []
p2_career_wins_last3_pct_before = []

# Player win percentage in last 5 matches before current match
p1_career_wins_last5_pct_before = []
p2_career_wins_last5_pct_before = []

# Player win percentage in last 10 matches before current match
p1_career_wins_last10_pct_before = []
p2_career_wins_last10_pct_before = []

# Define time periods for recent win percentages 
x1 = 3
x2 = 5
x3 = 10

for p1, p2 in zip(df_subset['p1_id'], df_subset['p2_id']):
    # Store total wins before this match
    p1_career_wins_before.append(career_wins_dict[p1])
    p2_career_wins_before.append(career_wins_dict[p2])

    # Get last 3 outcomes
    p1_last3 = career_wins_history_dict[p1][-x1:]
    p2_last3 = career_wins_history_dict[p2][-x1:]

    # Get last 5 outcomes
    p1_last5 = career_wins_history_dict[p1][-x2:]
    p2_last5 = career_wins_history_dict[p2][-x2:]

    # Get last 10 outcomes
    p1_last10 = career_wins_history_dict[p1][-x3:]
    p2_last10 = career_wins_history_dict[p2][-x3:]

    # Compute recent win 3 match win percentage
    p1_last3_pct = round((sum(p1_last3) / x1) * 100, 1) if len(p1_last3) == x1 else 0
    p2_last3_pct = round((sum(p2_last3) / x1) * 100, 1) if len(p2_last3) == x1 else 0

    # Compute recent 5 match win percentage
    p1_last5_pct = round((sum(p1_last5) / x2) * 100, 1) if len(p1_last5) == x2 else 0
    p2_last5_pct = round((sum(p2_last5) / x2) * 100, 1) if len(p2_last5) == x2 else 0

    # Compute recent 10 match win percentage
    p1_last10_pct = round((sum(p1_last10) / x3) * 100, 1) if len(p1_last10) == x3 else 0
    p2_last10_pct = round((sum(p2_last10) / x3) * 100, 1) if len(p2_last10) == x3 else 0

    # Append percentages to respective lists
    p1_career_wins_last3_pct_before.append(p1_last3_pct)
    p2_career_wins_last3_pct_before.append(p2_last3_pct)

    p1_career_wins_last5_pct_before.append(p1_last5_pct)
    p2_career_wins_last5_pct_before.append(p2_last5_pct)

    p1_career_wins_last10_pct_before.append(p1_last10_pct)
    p2_career_wins_last10_pct_before.append(p2_last10_pct)

    
    # Update total wins
    career_wins_dict[p1] += 1

    # Update recent win/loss history
    career_wins_history_dict[p1].append(1)  # p1 won
    career_wins_history_dict[p2].append(0)  # p2 lost

# Assign to DataFrame
df_subset['p1_career_wins_before'] = p1_career_wins_before
df_subset['p2_career_wins_before'] = p2_career_wins_before
df_subset['p1_career_wins_last3_pct_before'] = p1_career_wins_last3_pct_before
df_subset['p2_career_wins_last3_pct_before'] = p2_career_wins_last3_pct_before
df_subset['p1_career_wins_last5_pct_before'] = p1_career_wins_last5_pct_before
df_subset['p2_career_wins_last5_pct_before'] = p2_career_wins_last5_pct_before
df_subset['p1_career_wins_last10_pct_before'] = p1_career_wins_last10_pct_before
df_subset['p2_career_wins_last10_pct_before'] = p2_career_wins_last10_pct_before

# Removing Anomalous Data after ELOs and Match Histories have been Calculated

In [40]:
# Remove rows with NaN values for height, surface and age columns
df_subset = df_subset.dropna(subset=['p1_height'])
df_subset = df_subset.dropna(subset=['p2_height'])
df_subset = df_subset.dropna(subset=['surface'])
df_subset = df_subset.dropna(subset=['p1_age'])
df_subset = df_subset.dropna(subset=['p2_age'])

# Remove rows with unknown player handedness
df_subset = df_subset[(df_subset['p1_hand'] != 'U') & (df_subset['p2_hand'] != 'U') & (df_subset['p2_hand'] != 'A')]

#List of anomalous players to remove
anom_heights_list = ['Jorge Brian Panta Herreros',
'Johannes Ingildsen',
'Viacheslav Bielinskyi',
'Kooros Ghasemi',
'Alexander Stater',                     
'William Grant',
'Ilija Vucic',
'Andrew Rogers'
]

# remove anomalous players
mask = df_subset['p1_name'].isin(anom_heights_list) | df_subset['p2_name'].isin(anom_heights_list)
df_subset = df_subset.drop(df_subset[mask].index)

###### Checking for any NaN values or anomolous data

In [41]:
# Displayes number of NaN values for each column
df_subset.isna().sum()

tournament_date                              0
tournament_name                              0
tournament_level                             0
surface                                      0
p1_name                                      0
p1_id                                        0
p1_age                                       0
p1_height                                    0
p1_hand                                      0
p2_name                                      0
p2_id                                        0
p2_age                                       0
p2_height                                    0
p2_hand                                      0
p1_age_diff                                  0
p2_age_diff                                  0
p1_height_diff                               0
p2_height_diff                               0
p1_h2h_wins                                  0
p2_h2h_wins                                  0
p1_h2h_wins_before_total_diff                0
p2_h2h_wins_b

In [42]:
df_subset.describe()

,tournament_date,p1_id,p1_age,p1_height,p2_id,p2_age,p2_height,p1_age_diff,p2_age_diff,p1_height_diff,p2_height_diff,p1_h2h_wins,p2_h2h_wins,p1_h2h_wins_before_total_diff,p2_h2h_wins_before_total_diff,p1_h2h_wins_before_last1_diff,p2_h2h_wins_before_last1_diff,p1_h2h_wins_before_last2_diff,p2_h2h_wins_before_last2_diff,p1_h2h_wins_before_last3_diff,p2_h2h_wins_before_last3_diff,p1_h2h_wins_before_last4_diff,p2_h2h_wins_before_last4_diff,p1_h2h_wins_before_last5_diff,p2_h2h_wins_before_last5_diff,p1_h2h_wins_before_last10_diff,p2_h2h_wins_before_last10_diff,p1_elo_before,p2_elo_before,p1_elo_rolling_last5,p2_elo_rolling_last5,p1_elo_rolling_last10,p2_elo_rolling_last10,p1_elo_rolling_last20,p2_elo_rolling_last20,p1_elo_rolling_last5_diff_before,p2_elo_rolling_last5_diff_before,p1_elo_rolling_last10_diff_before,p2_elo_rolling_last10_diff_before,p1_elo_rolling_last20_diff_before,p2_elo_rolling_last20_diff_before,p1_elo_diff_before,p2_elo_diff_before,p1_surface_elo_before,p2_surface_elo_before,p1_surface_elo_rolling_last5,p2_surface_elo_rolling_last5,p1_surface_elo_rolling_last10,p2_surface_elo_rolling_last10,p1_surface_elo_rolling_last20,p2_surface_elo_rolling_last20,p1_surface_elo_rolling_last5_diff_before,p2_surface_elo_rolling_last5_diff_before,p1_surface_elo_rolling_last10_diff_before,p2_surface_elo_rolling_last10_diff_before,p1_surface_elo_rolling_last20_diff_before,p2_surface_elo_rolling_last20_diff_before,p1_surface_elo_diff_before,p2_surface_elo_diff_before,p1_total_matches_played_before,p2_total_matches_played_before,p1_total_matches_played_before_diff,p2_total_matches_played_before_diff,p1_career_wins_before,p2_career_wins_before,p1_career_wins_last3_pct_before,p2_career_wins_last3_pct_before,p1_career_wins_last5_pct_before,p2_career_wins_last5_pct_before,p1_career_wins_last10_pct_before,p2_career_wins_last10_pct_before
count,122158,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.00000,122158.00000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000,122158.000000
mean,2019-01-23 07:22:18.587730688,126843.787963,26.254729,185.716064,127245.235408,26.211628,185.301339,0.043102,-0.043102,0.414725,-0.414725,0.343342,0.267907,0.075435,-0.075435,0.040841,-0.040841,0.055486,-0.055486,0.063713,-0.063713,0.068027,-0.068027,0.070507,-0.070507,0.074756,-0.074756,1623.867876,1583.953937,1621.831317,1583.367016,1620.214837,1582.719077,1616.842286,1580.773146,38.464300,-38.464300,37.495760,-37.495760,36.069140,-36.069140,39.913939,-39.913939,1598.766196,1564.685105,1596.838594,1563.929673,1595.476031,1563.316287,1592.835508,1561.852928,32.908921,-32.908921,32.159744,-32.159744,30.98258,-30.98258,34.081092,-34.081092,196.626844,178.543206,18.083638,-18.083638,109.906179,96.409175,52.884359,50.211615,53.010364,48.813668,52.239640,47.164819
min,2012-01-01 00:00:00,100644.000000,14.900000,157.000000,100644.000000,14.800000,157.000000,-25.400000,-26.400000,-43.000000,-41.000000,0.000000,0.000000,-12.000000,-16.000000,-1.000000,-1.000000,-2.000000,-2.000000,-3.000000,-3.000000,-4.000000,-4.000000,-5.000000,-5.000000,-10.000000,-10.000000,1154.000000,1159.000000,1229.000000,1170.000000,1260.000000,1179.000000,12

# Reset Indexing

In [43]:
df_subset = df_subset.reset_index(drop=True)

# Creating Target Feature

###### In the data I am using, the winner of the match is always player 1. To model this data, the winner must be randomly player 1 or player 2 and a new 'result' feature needs to be added 

In [45]:
import numpy as np

# Make a copy so original data isn't affected
df_balanced = df_subset.copy()

# Add a result column and make the value equal to 1 because player 1 always wins
df_balanced["result"] = 1

# Randomly choose half of the rows to swap player1 and player2
swap_mask = np.random.rand(len(df_subset)) < 0.5

# Swap player-related columns where swap_mask is True
cols_to_swap = [
    ('p1_id', 'p2_id'),
    ('p1_hand', 'p2_hand'),
    ('p1_name', 'p2_name'),
    ('p1_age', 'p2_age'),
    ('p1_height', 'p2_height'),
    ('p1_age_diff', 'p2_age_diff'),
    ('p1_height_diff', 'p2_height_diff'),
    ('p1_h2h_wins', 'p2_h2h_wins'),
    ('p1_h2h_wins_before_total_diff', 'p2_h2h_wins_before_total_diff'),
    ('p1_h2h_wins_before_last1_diff', 'p2_h2h_wins_before_last1_diff'),
    ('p1_h2h_wins_before_last2_diff', 'p2_h2h_wins_before_last2_diff'),
    ('p1_h2h_wins_before_last3_diff', 'p2_h2h_wins_before_last3_diff'),
    ('p1_h2h_wins_before_last4_diff', 'p2_h2h_wins_before_last4_diff'),
    ('p1_h2h_wins_before_last5_diff', 'p2_h2h_wins_before_last5_diff'),
    ('p1_h2h_wins_before_last10_diff', 'p2_h2h_wins_before_last10_diff'),
    ('p1_elo_before', 'p2_elo_before'),
    ('p1_elo_diff_before', 'p2_elo_diff_before'),
    ('p1_surface_elo_before', 'p2_surface_elo_before'),
    ('p1_surface_elo_diff_before', 'p2_surface_elo_diff_before'),
    ('p1_total_matches_played_before', 'p2_total_matches_played_before'),
    ('p1_total_matches_played_before_diff', 'p2_total_matches_played_before_diff'),
    ('p1_career_wins_before', 'p2_career_wins_before'),
    ('p1_career_wins_last3_pct_before', 'p2_career_wins_last3_pct_before'),
    ('p1_career_wins_last5_pct_before', 'p2_career_wins_last5_pct_before'),
    ('p1_career_wins_last10_pct_before', 'p2_career_wins_last10_pct_before'),
    ('p1_elo_rolling_last5', 'p2_elo_rolling_last5'),
    ('p1_elo_rolling_last10', 'p2_elo_rolling_last10'),
    ('p1_elo_rolling_last20', 'p2_elo_rolling_last20'),
    ('p1_elo_rolling_last5_diff_before', 'p2_elo_rolling_last5_diff_before'),
    ('p1_elo_rolling_last10_diff_before', 'p2_elo_rolling_last10_diff_before'),
    ('p1_elo_rolling_last20_diff_before', 'p2_elo_rolling_last20_diff_before'),
    ('p1_surface_elo_rolling_last5', 'p2_surface_elo_rolling_last5'),
    ('p1_surface_elo_rolling_last10', 'p2_surface_elo_rolling_last10'),
    ('p1_surface_elo_rolling_last20', 'p2_surface_elo_rolling_last20'),
    ('p1_surface_elo_rolling_last5_diff_before', 'p2_surface_elo_rolling_last5_diff_before'),
    ('p1_surface_elo_rolling_last10_diff_before', 'p2_surface_elo_rolling_last10_diff_before'),
    ('p1_surface_elo_rolling_last20_diff_before', 'p2_surface_elo_rolling_last20_diff_before')
]

# Loop through each pair of columns and swap values for the selected rows
for col1, col2 in cols_to_swap:
    temp = df_balanced.loc[swap_mask, col1].copy()
    df_balanced.loc[swap_mask, col1] = df_balanced.loc[swap_mask, col2]
    df_balanced.loc[swap_mask, col2] = temp

# Update the result column to 0 where players were swapped
df_balanced.loc[swap_mask, "result"] = 0

# Removing Redundant Features

In [46]:
redundant_features = [
    'tournament_name',
    'p1_id',
    'p2_id',
    'p2_age_diff',
    'p2_height_diff',
    'p2_h2h_wins_before_total_diff',
    'p2_h2h_wins_before_last1_diff',
    'p2_h2h_wins_before_last2_diff',
    'p2_h2h_wins_before_last3_diff',
    'p2_h2h_wins_before_last4_diff',
    'p2_h2h_wins_before_last5_diff',
    'p2_h2h_wins_before_last10_diff',
    'p2_elo_diff_before',
    'p2_surface_elo_diff_before',
    'p2_total_matches_played_before_diff',
    'p2_career_wins_last3_pct_before',
    'p2_career_wins_last5_pct_before',
    'p2_career_wins_last10_pct_before'
]

# Drop them from the DataFrame
df_balanced = df_balanced.drop(redundant_features, axis=1)

# Encoding Players Names

In [48]:
from sklearn.preprocessing import LabelEncoder

# Collect all players names 
all_players = pd.concat([df_balanced['p1_name'], df_balanced['p2_name']])

# Fit a LabelEncoder on all names
le_players = LabelEncoder()
le_players.fit(all_players)

# Transform player name columns to player id columns
df_balanced['p1_id'] = le_players.transform(df_balanced['p1_name'])
df_balanced['p2_id'] = le_players.transform(df_balanced['p2_name'])

# Encode player hand
le_hand = {'R': 0, 'L': 1}
df_balanced['p1_hand_id'] = df_balanced['p1_hand'].map(le_hand)
df_balanced['p2_hand_id'] = df_balanced['p2_hand'].map(le_hand)
df_balanced['p1_hand_id'] = df_balanced['p1_hand_id'].astype(int)
df_balanced['p2_hand_id'] = df_balanced['p2_hand_id'].astype(int)

# Encode surface
le_surface = LabelEncoder()
df_balanced['surface_id'] = le_surface.fit_transform(df_balanced['surface'])

# Encode tournament level
le_tournament_level = LabelEncoder()
df_balanced['tournament_level_id'] = le_tournament_level.fit_transform(df_balanced['tournament_level'])

# Reordering Column Names

In [49]:
cols_new_order = [
    'tournament_date',
    'tournament_level_id',
    'surface_id',
    'p1_id',
    'p1_hand_id',
    'p2_id',
    'p2_hand_id',
    'p1_age_diff',
    'p1_height_diff',
    'p1_h2h_wins_before_total_diff',
    'p1_elo_diff_before',
    'p1_elo_rolling_last5_diff_before',
    'p1_elo_rolling_last10_diff_before',
    'p1_elo_rolling_last20_diff_before',
    'p1_surface_elo_diff_before',
    'p1_surface_elo_rolling_last5_diff_before',
    'p1_surface_elo_rolling_last10_diff_before',
    'p1_surface_elo_rolling_last20_diff_before',
    'p1_total_matches_played_before_diff',
    'result'
]
df_balanced = df_balanced[cols_new_order].copy()

# Saving Dataframe as a new .csv file

In [50]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122158 entries, 0 to 122157
Data columns (total 20 columns):
 #   Column                                     Non-Null Count   Dtype         
---  ------                                     --------------   -----         
 0   tournament_date                            122158 non-null  datetime64[ns]
 1   tournament_level_id                        122158 non-null  int32         
 2   surface_id                                 122158 non-null  int32         
 3   p1_id                                      122158 non-null  int32         
 4   p1_hand_id                                 122158 non-null  int32         
 5   p2_id                                      122158 non-null  int32         
 6   p2_hand_id                                 122158 non-null  int32         
 7   p1_age_diff                                122158 non-null  float64       
 8   p1_height_diff                             122158 non-null  float64       
 9   p1_h

In [44]:
df_balanced.to_csv('feature_engineered_tennis_data.csv', index = False)